<img src="http://landlab.github.io/assets/Landlab-logo.png"
style="float:left;width:150px;padding:0px">  

# Lowering the barriers to computational modeling of Earth’s surface: coupling Jupyter Notebooks with Landlab, HydroShare, and CyberGIS for research and education <br /> 
<br /> 
### Use Case 2: Gridded Hydrometeorology forcing a Cellular Automaton Vegetation-Ecohydrologic Model
* Load data from a Landlab model published on HydroShare. <br />
* Define a geographic subset (New Mexico) within the North America.
* Calculate mean climatology for a range of elevation bands.
* Run the Landlab ecohydrology model for a watershed given various climate scenarios.<br />
* Explore sensitivity to climate by comparing the watershed model results.
* Save results to a new HydroShare resource.  <br />

## Sai- we need a citation here and to update the list below.  <br />

#### This research is actively being developed by the Watershed Dynamics Research Group in the Civil and Environmental Engineering Department at the University of Washington for the Thunder Creek basin in the Skagit Watershed, WA, USA in collaboration with the HydroShare team, CUAHSI and NCSA partners.
 <br /> <img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:left;width:120px;padding:10px">   
<img src="http://www.ncsa.illinois.edu/assets/img/logos_ncsa.png"
 style="float:right;width:120px;padding:10px"> 
<img src="https://www.cuahsi.org/assets/images/logo.png">

<hr>
<small> For instructions on how to run an interactive iPython notebook, click here: <a href="https://github.com/landlab/tutorials/blob/master/README.md">https://github.com/landlab/tutorials/blob/master/README.md</a></small><br>
<small>For the unexpanded version to download and run, click here: <a href="http://nbviewer.jupyter.org/github/landlab/tutorials/blob/master/ecohydrology/cellular_automaton_vegetation_DEM/cellular_automaton_vegetation_DEM_unexpanded.ipynb">http://nbviewer.jupyter.org/github/landlab/tutorials/blob/master/ecohydrology/cellular_automaton_vegetation_DEM/cellular_automaton_vegetation_DEM_unexpanded.ipynb</a></small><br>
<small>For more Landlab tutorials, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

This tutorial demonstrates implementation of the Cellular Automaton Tree-GRass-Shrub Simulator (CATGRaSS) [Zhou et al., 2013] on a flat domain. This model is built using components from the Landlab component library. CATGRaSS is spatially explicit model of plant coexistence. It simulates local ecohydrologic dynamics (soil moisture, transpiration, biomass) and spatial evolution of tree, grass, and shrub Plant Functional Types (PFT) driven by rainfall and solar radiation. 

Each cell in the model grid can hold a single PFT or remain empty. Tree and shrub plants disperse seeds to their neighbors. Grass seeds are assumed to be available at each cell. Establishment of plants in empty cells is determined probabilistically based on water stress of each PFT. Plants with lower water stress have higher probability of establishment. Plant mortality is simulated probabilistically as a result of aging and drought stress. Fires and grazing will be added to this model soon.  

This model (driver) contains:
  - A local vegetation dynamics model that simulates storm and inter-storm water balance and ecohydrologic fluxes (ET, runoff), and plant biomass dynamics by coupling the following components:
        - PrecipitationDistribution
        - Radiation
        - PotentialEvapotranspiration
        - SoilMoisture
        - Vegetation

  - A spatially explicit probabilistic cellular automaton component that simulates plant competition by tracking establishment and mortality of plants based on soil moisture stress:
        - VegCA
    
To run this Jupyter notebook, please make sure that the following files are in the same folder:
        - cellular_automaton_vegetation_flat_domain.ipynb (this notebook)
        - Inputs_Vegetation_CA.txt (Input parameters for the model)
        - Ecohyd_functions_flat.py (Utility functions)

[Ref: Zhou, X, E. Istanbulluoglu, and E.R. Vivoni. "Modeling the ecohydrological role of aspect-controlled radiation on tree-grass-shrub coexistence in a semiarid climate." Water Resources Research 49.5 (2013): 2872-2895]


In this tutorial, we are going to work with a landscape in central New Mexico, USA, where aspect controls the organization of PFTs. The climate in this area is semi-arid with Mean Annual Precipitation (MAP) of 254 mm [Zhou et. al 2013]. 
We will do the following: 
- Import a landscape 
- Initialize the landscape with random distribution of PFTs
- Run the coupled Ecohydrology and cellular automata plant competition model for 50 years
- Visualize and examine outputs

## 1.  Notebook Setup and Preparation

To run this notebook, we must import several libraries.
The hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading. and creation. Additional libraries support the functions of Landlab.
### 1a.  Import libraries installed on CUAHSI JupyterHub server

In [ ]:
#Python libraries available on CUAHSI JupyterHub 
import os, json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import warnings 
warnings.filterwarnings('ignore')

#HydroShare Utilities
from utilities import hydroshare
hs=hydroshare.hydroshare()

### Set variables for interacting with HydroShare from this notebook
<br />
The home directory on the JupyterHub server is printed. You can navigate to see the data using a terminal or folders. 

### 1b.  Import python scripts from a Github repository

#### Please see the [Observatory]( https://github.com/ChristinaB/Observatory/blob/master/README.md) repository on Github with a Readme instructions on how to use Git and the JupyterHub server terminal to push/pull changes.   After completing the steps to get the observatory_gridded_hydromet.py script into your HydroShare Utilities folder, execute the next cell. 

In [ ]:
import ogh

In [ ]:
homedir = ogh.mapContentFolder(str(os.environ["HS_RES_ID"]))
print('Data will be loaded from and save to:'+homedir)

## 2. Download and generate lists of gridded climate points for a watershed
Retrieve a list of grid points and configuration file from a HydroShare resource
This example uses a ascii text that is stored in HydroShare at the following url: https://www.hydroshare.org/resource/d90289409f904017831d308642c1eb30/ . The data for our processing routines can be retrieved using the getResourceFromHydroShare function by passing in the global identifier from the url above.  In the next cell, we download this resource from HydroShare, and identify that the table in this resource is the 'mappingfile' variable identifying the Lat/Long points to be used for downloading hydrometeorology data.  The file must include columns with station numbers (this can be aribitrary), latitude, longitude, and elevation. The header of these columns must be FID, LAT, LONG_, and ELEV or RASTERVALU, respectively. The station numbers will be used for the remainder of the code to uniquely reference data from each climate station, as well as to identify minimum, maximum, and average elevation of all of the climate stations.

### Download areas of interest from HydroShare Resources

In [ ]:
# This resource type is Composite Resource containing a Geographic Feature (ESRI Shapefile). 
hs.getResourceFromHydroShare('5b401852758e4d49a87a788c1008b027')
download_extent_shapefile = hs.content['New_Mexico.shp']

In [ ]:
### TreatGeoSelf to a mapping tool that uses your shapefile
# Reuse this exising point shapefile of available 1/16 degree grid centroid locations 
# shared on HydroShare as a public resource
hs.getResourceFromHydroShare('ef2d82bf960144b4bfb1bae6242bcc7f')
NAmer = hs.content['NAmer_dem_list.shp']

# Read in the Observatory metadata file 
This file contains the variables, data types and metadata related to Livneh et al., 2013; 2015 and Salathe et al., 2014 gridded hydrometeorology products. 

In [ ]:
#Assuming this is pulled from Github, how can we import this from Utilities.
#Otherwise it needs to be in each HydroShare resource - which if fine too. 
# initialize ogh_meta data file
meta_file = dict(ogh.ogh_meta())
sorted(meta_file.keys())

## TreatGeoSelf() to a easily generated list of lat/long points in your area of interest

In [ ]:
%%time
#New Mexico Extent is approximately 7900 gridded cells
mappingfile = ogh.treatgeoself(shapefile=download_extent_shapefile, 
                               NAmer=NAmer, 
                               buffer_distance=0,
                               mappingfile=os.path.join(homedir,'new_mexico_gridmet.csv'))
print(mappingfile)

In [ ]:
ogh.griddedCellGradient(mappingfile=mappingfile, 
                        shapefile=download_extent_shapefile,
                        outfilepath=os.path.join(homedir, 'NM_elev_gradient.png'),
                        plottitle='New Mexico watershed\nGridded cell elevation gradient',
                        colorbar_label='Average elevation (meters)',
                        spatial_resolution=1/16, margin=0.25, epsg=3857, column='ELEV',
                        basemap_image='Elevation/World_Hillshade', cmap='terrain')

 ## 3. Download gridded climate data 
 ### Get Daily Meteorologic Data from  Livneh et al. 2015 (1950-2013; North America extent)

Please cite 2015 data with: <br/>
Livneh B., T.J. Bohn, D.S. Pierce, F. Munoz-Ariola, B. Nijssen, R. Vose, D. Cayan, and L.D. Brekke, 2015: A spatially comprehensive, hydrometeorological data set for Mexico, the U.S., and southern Canada 1950-2013, Nature Scientific Data, 5:150042, doi:10.1038/sdata.2015.42.

In [ ]:
# %%time
# # uncomment and run if the files have already been downloaded
# ogh.remapCatalog(homedir=homedir, 
#                  mappingfile=mappingfile, 
#                  subdir='livneh2015/Daily_MET_1950_2013/raw', 
#                  catalog_label='dailymet_livneh2015')

In [ ]:
%%time
# From the reference mapping file, read-in, download, and unzip the data files for the longitude and latitude points
ogh.getDailyMET_livneh2015(homedir, mappingfile)

In [ ]:
t1 = ogh.mappingfileSummary(listofmappingfiles = [mappingfile], 
                            listofwatershednames = ['New Mexico'],
                            meta_file=meta_file)

t1

## 4.  Create gridded climate data dictionary

### Make a dictionary of the original time series from Livneh and WRF, the monthly average statistics for each grid cell will be used to calculate differences between the two datasets.

In [ ]:
min_elev=0
max_elev=1500

ltm_low = ogh.gridclim_dict(mappingfile=mappingfile,
                            dataset='dailymet_livneh2015',
                            metadata=meta_file,
                            min_elev=min_elev,
                            max_elev=max_elev)

ogh.saveDictOfDf(dictionaryObject=ltm_low, outfilepath=os.path.join(homedir,'NewMexico_lowElevation.json'))

sorted(ltm_low.keys())
del ltm_low

In [ ]:
min_elev=1500
max_elev=2500

ltm_mid = ogh.gridclim_dict(mappingfile=mappingfile,
                            dataset='dailymet_livneh2015',
                            metadata=meta_file,
                            min_elev=min_elev,
                            max_elev=max_elev)

ogh.saveDictOfDf(dictionaryObject=ltm_mid, outfilepath=os.path.join(homedir,'NewMexico_midElevation.json'))

sorted(ltm_mid.keys())
del ltm_mid

In [ ]:
min_elev=2500
max_elev=3500

ltm_high = ogh.gridclim_dict(mappingfile=mappingfile,
                             dataset='dailymet_livneh2015',
                             metadata=meta_file,
                             min_elev=min_elev,
                             max_elev=max_elev)

ogh.saveDictOfDf(dictionaryObject=ltm_high, outfilepath=os.path.join(homedir,'NewMexico_highElevation.json'))

sorted(ltm_high.keys())
del ltm_high

In [ ]:
min_elev=0
max_elev=3500

ltm_all = ogh.gridclim_dict(mappingfile=mappingfile,
                            dataset='dailymet_livneh2015',
                            metadata=meta_file,
                            min_elev=min_elev,
                            max_elev=max_elev)

ogh.saveDictOfDf(dictionaryObject=ltm_all, outfilepath=os.path.join(homedir,'NewMexico_allElevation.json'))

sorted(ltm_all.keys())
del ltm_all

#### Select a Dictionary and convert meteorology data from pandas dataframe into numpy arrays